# TESS Atlas fit for TOI {{{TOINUMBER}}}

**Version: {{{VERSIONNUMBER}}}**

**Note: This notebook was automatically generated as part of the TESS Atlas project. More information can be found on GitHub:** [github.com/dfm/tess-atlas](https://github.com/dfm/tess-atlas)

In this notebook, we do a quicklook fit for the parameters of the TESS Objects of Interest (TOI) in the system number {{{TOINUMBER}}}.
To do this fit, we use the [exoplanet](https://exoplanet.dfm.io) library and you can find more information about that project at [exoplanet.dfm.io](https://exoplanet.dfm.io).

From here, you can scroll down and take a look at the fit results, or you can:

- [open the notebook in Google Colab to run the fit yourself](https://colab.research.google.com/github/dfm/tess-atlas/blob/gh-pages/notebooks/{{{VERSIONNUMBER}}}/toi-{{{TOINUMBER}}}.ipynb),
- [view the notebook on GitHub](https://github.com/dfm/tess-atlas/blob/gh-pages/notebooks/{{{VERSIONNUMBER}}}/toi-{{{TOINUMBER}}}.ipynb), or
- [download the notebook](https://github.com/dfm/tess-atlas/raw/gh-pages/notebooks/{{{VERSIONNUMBER}}}/toi-{{{TOINUMBER}}}.ipynb).



## Caveats

There are many caveats associated with this relatively simple "quicklook" type of analysis that should be kept in mind.
Here are some of the main things that come to mind:

1. The orbits that we fit are constrained to be *circular*. One major effect of this approximation is that the fit will significantly overestimate the confidence of the impact parameter constraint, so the results for impact parameter shouldn't be taken too seriously. 

2. Transit timing variations, correlated noise, and (probably) your favorite systematics are ignored. Sorry!

3. This notebook was generated automatically without human intervention. Use at your own risk!

## Table of Contents

1. [Getting started](#Getting-started)
2. [Data & de-trending](#Data-%26amp%3B-de-trending)
3. [Removing stellar variability](#Removing-stellar-variability)
4. [Transit model in PyMC3 & exoplanet](#Transit-model-in-PyMC3-%26amp%3B-exoplanet)
5. [Sampling](#Sampling)
6. [Posterior constraints](#Posterior-constraints)
7. [Attribution](#Attribution)

## Getting started

To get going, we'll need to make out plots show up inline and install a few packages:

In [ ]:
%matplotlib inline
!pip install -q -U lightkurve fbpca exoplanet corner pymc3 dynesty isochrones

Then we'll set up the plotting styles and do all of the imports:

In [ ]:
import functools
import logging
import multiprocessing as mp
import os
import warnings
from pathlib import Path
from typing import List
from copy import deepcopy

import arviz as az
import corner
import exoplanet as xo
import lightkurve as lk
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import theano.tensor as tt
from IPython.display import display
from astroquery.mast import Catalogs

get_ipython().magic('config InlineBackend.figure_format = "retina"')

# TEMPORARY WORKAROUND
mp.set_start_method("fork")

# Don't use the schmantzy progress bar
os.environ["EXOPLANET_NO_AUTO_PBAR"] = "true"

# Warning
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Logging setup
logger = logging.getLogger("theano.gof.compilelock")
logger.setLevel(logging.ERROR)
logger = logging.getLogger("exoplanet")
logger.setLevel(logging.DEBUG)

# matplotlib settings
plt.style.use("default")
plt.rcParams["savefig.dpi"] = 100
plt.rcParams["figure.dpi"] = 100
plt.rcParams["font.size"] = 16
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = ["Liberation Sans"]
plt.rcParams["font.cursive"] = ["Liberation Sans"]
plt.rcParams["mathtext.fontset"] = "custom"
plt.rcParams['image.cmap'] = 'inferno'


# Constants
TOI_DATASOURCE = (
    "https://exofop.ipac.caltech.edu/tess/download_toi.php?sort=toi&output=csv"
)

In [ ]:
TOI_NUMBER = {{{TOINUMBER}}}
__version__ = {{{VERSIONNUMBER}}}
FILENAME = {{{FILENAME}}}

## Fitting stellar parameters

Next, we define some code to grab the TOI list from [ExoFOP](https://exofop.ipac.caltech.edu/tess/) to get the information about the system.

We wrap the information in three objects, a `TIC Entry`, a `Planet Candidate` and finally a `Lightcurve Data` object.

- The `TIC Entry` object holds one or more `Planet Candidate`s (each candidate associated with one TOI id number) and a `Lightcurve Data` for associated with the candidates. Note that the `Lightcurve Data` object is initially the same fopr each candidate but may be masked according to the candidate transient's period.

- The `Planet Candidate` holds informaiton on the TOI data collected (eg transit period, etc)

- The `Lightcurve Data` holds the lightcurve time and flux data for the planet candidates.

In [ ]:
def get_tic_data_from_database(toi_number: int) -> pd.DataFrame:
    """Get rows of about a TIC  from ExoFOP associated with a TOI target.
    :param int toi_number: The TOI number for which the TIC data is obtained
    :return: Dataframe with all TOIs for the TIC which contains TOI {toi_id}
    :rtype: pd.DataFrame
    """
    tois = pd.read_csv(TOI_DATASOURCE)
    toi = tois[tois["TOI"] == toi_number + 0.01].iloc[0]
    tic = toi["TIC ID"]
    tois_for_tic = tois[tois["TIC ID"] == tic].sort_values("TOI")
    if len(tois_for_tic) < 1:
        raise ValueError(f"TOI-{toi_number} data for TIC-{tic} does not exist.")
    return tois_for_tic


In [ ]:
class PlanetCandidate:
    """Plant Candidate obtained by TESS."""

    def __init__(self, toi_id: float, period: float, t0: float, depth: float,
                 duration: float):
        """
        :param float toi_id: The toi number X.Y where the Y represents the TOI sub number
        :param float period: Planet candidate orbital period (in days)
        :param float t0: Epoch (timestamp) of the primary transit in Barycentric Julian Date
        :param float depth: Planet candidate transit depth, in parts per million
        :param float duration: Planet candidate transit duration, in days.
        """
        self.toi_id = toi_id
        self.period = period
        self.t0 = t0
        self.depth = depth
        self.duration = duration

    @classmethod
    def from_toi_database_entry(cls, toi_data: dict):
        return cls(
            toi_id=toi_data['TOI'],
            period=toi_data["Period (days)"],
            t0=toi_data["Epoch (BJD)"] - 2457000,  # convert to TBJD
            depth=toi_data["Depth (ppm)"] * 1e-3,  # convert to parts per thousand
            duration=toi_data["Duration (hours)"] / 24.0,  # convert to days
        )

    def to_dict(self):
        return {
            "TOI": self.toi_id,
            "Period (days)": self.period,
            "Epoch (TBJD)": self.t0,
            "Depth (ppt)": self.depth,
            "Duration (days)": self.duration
        }

In [ ]:
class LightcurveData:
    """Stores Light Curve data for a single target"""

    def __init__(self, time:np.ndarray, flux:np.ndarray, flux_err:np.ndarray):
        """
        :param np.ndarray time: The time in days.
        :param np.ndarray flux: The relative flux in parts per thousand.
        :param np.ndarray fluex_err: The flux err in parts per thousand.
        """
        self.time = time
        self.flux = flux
        self.flux_err = flux_err
        self.masked = False

    @classmethod
    def from_mast(cls, tic: int):
        """Uses lightkurve to get TESS data for a TIC from MAST"""
        search = lk.search_lightcurve(target=f'TIC {tic}', mission="TESS")
        print(f"Downloading {len(search)} observations of lightcurve data (TIC {tic})")
        data = search.download_all()
        print("Completed lightcurve data download")
        data = data.stitch()
        data = data.remove_nans().remove_outliers()
        return cls(
            time=np.ascontiguousarray(data.time.value, dtype=np.float64),
            flux=np.ascontiguousarray(1e3 * (data.flux.value - 1), dtype=np.float64),
            flux_err=np.ascontiguousarray(1e3 * data.flux_err.value, dtype=np.float64)
        )

    def mask(self, days: float, t0_guess:float, period_guess:float):
        """Mask lightcurce data to look only at the central "days" duration of data """
        if self.masked:
            raise ValueError("Lightcurve already masked once.")
        period_buffer = 0.5 * period_guess
        area =  (self.time - t0_guess + period_buffer) % period_guess - period_buffer
        transit_mask = np.abs(area) < days
        self.time = np.ascontiguousarray(self.time[transit_mask])
        self.flux = np.ascontiguousarray(self.flux[transit_mask])
        self.flux_err = np.ascontiguousarray(self.flux_err[transit_mask])
        self.masked = True


In [ ]:

class TicEntry:
    """Hold information about a TIC (TESS Input Catalog) entry"""

    def __init__(self, tic: int, candidate_list: List[PlanetCandidate]):
        self.tic_number = tic
        self.candidate_list = candidate_list
        self.candidate_lightcurves = []

    @property
    def planet_count(self):
        return len(self.candidate_list)

    @classmethod
    def generate_tic_from_toi_number(cls, toi: int):
        tois_for_tic_table = get_tic_data_from_database(toi)
        candidates = []
        for index, toi_data in tois_for_tic_table.iterrows():
            candidate = PlanetCandidate.from_toi_database_entry(toi_data.to_dict())
            candidates.append(candidate)
        return cls(
            tic=int(tois_for_tic_table['TIC ID'].iloc[0]),
            candidate_list=candidates,
        )

    def load_lightcurve(self):
        lc = LightcurveData.from_mast(tic=self.tic_number)
        self.candidate_lightcurve = [deepcopy(lc) for _ in range(self.planet_count)]

    def mask_lightcurves(self, days:float):
        """Mask all candidate lightcurves"""
        for i in range(self.planet_count):
            self.candidate_lightcurve[i].mask(
                days=days,
                t0_guess=self.candidate_list[i].t0,
                period_guess=self.candidate_list[i].period,
            )
        
    def to_dataframe(self):
        return pd.DataFrame([candidate.to_dict() for candidate in self.candidate_list])

    def display(self):
        df = self.to_dataframe()
        df = df.transpose()
        df.columns = df.loc['TOI']
        display(df)

    def setup_outdir(self, version):
        toi = int(self.candidate_list[0].toi_id)
        output_dir = os.path.join("results", version, toi)
        os.makedirs(output_dir, exist_ok=True)
        self.outdir = output_dir

In [ ]:
tic_entry = TicEntry.generate_tic_from_toi_number(toi=TOI_NUMBER)
tic_entry.display()

Now, lets download and plot the TESS light curve data for the `Planet Candidate`s using [lightkurve](https://docs.lightkurve.org/):

In [ ]:
tic_entry.load_lightcurve()

In [ ]:
def plot_lightcurve(lc: LightcurveData):
    plt.plot(lc.time, lc.flux, "k", linewidth=0.5)
    plt.xlabel("time [days]")
    plt.ylabel("relative flux [ppt]");

In [ ]:
plot_lightcurve(tic_entry.candidate_lightcurve[0])

For efficiency purposes, let's extract just the data within 0.25 days of the transits:

In [ ]:
days = 0.25
tic_entry.mask_lightcurves(days)

In [ ]:
def plot_masked_lightcurve_flux_vs_time_since_transit(tic_entry:TicEntry, days: float):
    num_planets = tic_entry.planet_count
    fig, axs = plt.subplots(num_planets, figsize=(8, 4 * num_planets), sharex=True, squeeze=0)
    for i, ax in enumerate(axs.flat):
        lc = tic_entry.candidate_lightcurve[i]
        planet = tic_entry.candidate_list[i]
        period_bufer = 0.5 * planet.period
        time_fold = (lc.time - planet.t0 + period_bufer) % planet.period - period_bufer
        ax.set_title(f"TOI-{planet.toi_id:.2f}")
        im = ax.scatter(time_fold, lc.flux, c=lc.time, s=3, alpha=0.1)
        ax.set_ylabel("Relative Flux [ppt]")
        plt.colorbar(im, ax=axs[i],label="Time [days]")
    # axs[-1].set_xlabel("Time since transit [days]")
    # axs[-1].set_xlim(-days, days)

In [ ]:
plot_masked_lightcurve_flux_vs_time_since_transit(tic_entry, days)

That looks a little janky, but it's good enough for now.

## The probabilistic model

Here's how we set up the PyMC3 model in this case:

In [ ]:
def build_planet_transient_model(planet_candidate:PlanetCandidate, lc:LightcurveData):
    with pm.Model() as planet_transient_model:
        # Stellar parameters
        mean = pm.Normal("mean", mu=0.0, sigma=10.0)
        u = xo.distributions.QuadLimbDark("u")
        star_params = [mean, u]

        # Gaussian process noise model priors
        sigma = pm.InverseGamma("sigma", alpha=3.0, beta=2 * np.median(lc.flux_err))
        log_Sw4 = pm.Normal("log_Sw4", mu=0.0, sigma=10.0)
        log_w0 = pm.Normal("log_w0", mu=np.log(2 * np.pi / 10.0), sigma=10.0)
        kernel = xo.gp.terms.SHOTerm(log_Sw4=log_Sw4, log_w0=log_w0, Q=1.0 / 3)
        noise_params = [sigma, log_Sw4, log_w0]

        # Planet parameters priors
        log_ror = pm.Normal(
            "log_ror", mu=0.5 * np.log(planet_candidate.depth * 1e-3), sigma=10.0
        )
        ror = pm.Deterministic("ror", tt.exp(log_ror))

        # Orbital parameters priors
        log_period = pm.Normal("log_period", mu=np.log(planet_candidate.period), sigma=1.0)
        t0 = pm.Normal("t0", mu=planet_candidate.t0, sigma=1.0)
        log_dur = pm.Normal("log_dur", mu=np.log(0.1), sigma=10.0)
        b = xo.distributions.ImpactParameter("b", ror=ror)

        period = pm.Deterministic("period", tt.exp(log_period))
        dur = pm.Deterministic("dur", tt.exp(log_dur))

        # Set up the orbit
        orbit = xo.orbits.KeplerianOrbit(period=period, duration=dur, t0=t0, b=b)

        # We're going to track the implied density for reasons that will become clear later
        pm.Deterministic("rho_circ", orbit.rho_star)  # rho circ is part of the model

        # Set up the mean transit model
        star = xo.LimbDarkLightCurve(u)

        def lc_model(t):
            return mean + 1e3 * tt.sum(
                star.get_light_curve(orbit=orbit, r=ror, t=t), axis=-1
            )

        # Finally the GP observation model
        gp = xo.gp.GP(kernel, lc.time, lc.flux_err ** 2 + sigma ** 2, mean=lc_model)
        gp.marginal("obs", observed=lc.flux)  # compute likelihood (observed == data)

        # Double check that everything looks good - we shouldn't see any NaNs!
        print("Model test point:")
        print(planet_transient_model.check_test_point())

        # cache params
        params = dict(
            sigma=sigma,
            log_ror=log_ror,
            b=b,
            log_dur=log_dur,
            noise_params=noise_params,
            star_params=star_params,
        )

        return planet_transient_model, params, lc_model, gp


def optimize_model(model, sigma, log_ror, b, log_dur, noise_params, star_params):
    print("Optimizing model")
    with model:
        map_soln = model.test_point
        map_soln = xo.optimize(map_soln, [sigma])
        map_soln = xo.optimize(map_soln, [log_ror, b, log_dur])
        map_soln = xo.optimize(map_soln, noise_params)
        map_soln = xo.optimize(map_soln, star_params)
        map_soln = xo.optimize(map_soln)
        return map_soln

In [ ]:
planet_transient_model, params, lc_model, gp = build_planet_transient_model(
    planet_candidate=tic_entry.candidate_list[0],
    lc=tic_entry.candidate_lightcurve[0]
)
map_soln = optimize_model(planet_transient_model, **params)

Now we can plot our initial model:

In [ ]:
def plot_model(lc, planet_transient_model, map_soln, gp_pred, lc_pred):
    plt.figure(figsize=(8, 4))
    time_fold = (lc.time - map_soln["t0"] + 0.5 * map_soln["period"]) % map_soln[
        "period"
    ] - 0.5 * map_soln["period"]
    inds = np.argsort(time_fold)
    plt.scatter(time_fold, lc.flux - gp_pred - map_soln["mean"], c=lc.time, s=3)
    plt.plot(time_fold[inds], lc_pred[inds] - map_soln["mean"], "k")
    plt.xlabel("time since transit [days]")
    plt.ylabel("relative flux [ppt]")
    plt.colorbar(label="time [days]")
    plt.xlim(-days, days)


def obtain_prediction(model, gp, lc_model, map_soln, lc):
    with model:
        gp_pred, lc_pred = xo.eval_in_model([gp.predict(), lc_model(lc.time)], map_soln)
        return gp_pred, lc_pred

In [ ]:
gp_pred, lc_pred = obtain_prediction(planet_transient_model, gp, lc_model, map_soln, tic_entry.candidate_lightcurve[0])

In [ ]:
plot_model(tic_entry.candidate_lightcurve[0], planet_transient_model, map_soln, gp_pred, lc_pred)

That looks better!

Now on to sampling:

In [ ]:
TUNE = 1
DRAWS = 1
CHAINS = 1

np.random.seed(286923464)


def start_model_sampling(model):
    with model:
        trace = pm.sample(
            tune=TUNE,
            draws=DRAWS,
            start=map_soln,
            chains=CHAINS,
            cores=1,
            step=xo.get_dense_nuts_step(target_accept=0.9),
        )
        return trace

In [ ]:
trace = start_model_sampling(planet_transient_model)

Then we can take a look at the summary statistics:

In [ ]:
pm.summary(trace)

And plot the posterior covariances compared to the values from [Pepper et al. (2019)](https://arxiv.org/abs/1911.05150):

In [ ]:
def plot_posteriors(trace):
    samples = pm.trace_to_dataframe(trace, varnames=["period", "ror", "b"])
    corner.corner(samples);

In [ ]:
plot_posteriors(trace)

Finally, we save the posteriors and sampling metadata for future use.

In [ ]:
def validate_trace_filename(filename):
    suffix = Path(filename).suffix
    if suffix != ".netcdf":
        raise ValueError(f"{suffix} is an invalid extension.")

def save_trace(trace, filename):
    """Save pymc3 trace as a netcdf file"""
    validate_trace_filename(filename)
    az_trace = az.from_pymc3(trace)
    az_trace.to_netcdf(filename)

def load_trace(filename):
    """Load pymc3 trace from netcdf file and return an arviz InferenceData object"""
    validate_trace_filename(filename)
    return az.from_netcdf(filename)

In [ ]:
trace_filename = os.path.basename(FILENAME.replace(".ipynb", ".netcdf"))
save_trace(trace, trace_filename)

## Bonus: eccentricity

As discussed above, we fit this model assuming a circular orbit which speeds things up for a few reasons.
First, setting eccentricity to zero means that the orbital dynamics are much simpler and more computationally efficient, since we don't need to solve Kepler's equation numerically.
But this isn't actually the main effect!
Instead the bigger issues come from the fact that the degeneracies between eccentricity, arrgument of periasteron, impact parameter, and planet radius are hard for the sampler to handle, causing the sampler's performance to plummet.
In this case, by fitting with a circular orbit where duration is one of the parameters, everything is well behaved and the sampler runs faster.

But, in this case, the planet *is* actually on an eccentric orbit, so that assumption isn't justified.
It has been recognized by various researchers over the years (I first learned about this from [Bekki Dawson](https://arxiv.org/abs/1203.5537)) that, to first order, the eccentricity mainly just changes the transit duration.
The key realization is that this can be thought of as a change in the impled density of the star.
Therefore, if you fit the transit using stellar density (or duration, in this case) as one of the parameters (*note: you must have a* different *stellar density parameter for each planet if there are more than one*), you can use an independent measurement of the stellar density to infer the eccentricity of the orbit after the fact.
All the details are described in [Dawson & Johnson (2012)](https://arxiv.org/abs/1203.5537), but here's how you can do this here using the stellar density listed in the TESS input catalog:

In [ ]:
def plot_reweighted_ecentricity_samples(tic_number, trace):
    star = Catalogs.query_object(f"TIC {tic_number}", catalog="TIC", radius=0.001)
    tic_rho_star = float(star["rho"]), float(star["e_rho"])
    print("rho_star = {0} ± {1}".format(*tic_rho_star))

    # Extract the implied density from the fit
    rho_circ = np.repeat(trace["rho_circ"], 100)

    # Sample eccentricity and omega from their priors (the math might
    # be a little more subtle for more informative priors, but I leave
    # that as an exercise for the reader...)
    ecc = np.random.uniform(0, 1, len(rho_circ))
    omega = np.random.uniform(-np.pi, np.pi, len(rho_circ))

    # Compute the "g" parameter from Dawson & Johnson and what true
    # density that implies
    g = (1 + ecc * np.sin(omega)) / np.sqrt(1 - ecc ** 2)
    rho = rho_circ / g ** 3

    # Re-weight these samples to get weighted posterior samples
    log_weights = -0.5 * ((rho - tic_rho_star[0]) / tic_rho_star[1]) ** 2
    weights = np.exp(log_weights - np.max(log_weights))

    # Estimate the expected posterior quantiles
    q = corner.quantile(ecc, [0.16, 0.5, 0.84], weights=weights)
    print("eccentricity = {0:.2f} +{1[1]:.2f} -{1[0]:.2f}".format(q[1], np.diff(q)))

    corner.corner(
        np.vstack((ecc, omega)).T,
        weights=weights,
        plot_datapoints=False,
        labels=["eccentricity", "omega"],
    );

In [ ]:
plot_reweighted_ecentricity_samples(tic_entry.tic_number, trace)

As you can see, this eccentricity estimate is consistent (albeit with large uncertainties) with the value that [Pepper et al. (2019)](https://arxiv.org/abs/1911.05150) measure using radial velocities and it is definitely clear that this planet is not on a circular orbit.

## Citations

As described in the :ref:`citation` tutorial, we can use :func:`exoplanet.citations.get_citations_for_model` to construct an acknowledgement and BibTeX listing that includes the relevant citations for this model.

In [ ]:
with planet_transient_model:
    txt, bib = xo.citations.get_citations_for_model()
print(txt)

In [ ]:
print("\n".join(bib.splitlines()[:10]) + "\n...")